In [1]:
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
import matplotlib.pylab as plt
import seaborn as sns
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report

/home/yarrdopanas/miniconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
# read data

df_jog = pd.read_csv('jog_sub_1.csv')[:1500]
df_sit = pd.read_csv('sit_sub_1.csv')[:1500]

In [5]:
# add labels

df_jog['label'] = 1
df_sit['label'] = 0

In [6]:
# combine to the common df

df = pd.concat([df_jog, df_sit])

In [7]:
df.reset_index(inplace=True, drop=True)

In [8]:
df.shape

(3000, 14)

In [9]:
df['id'] = df.index//15

In [10]:
y = np.zeros(200)
y[100:] = 1

In [11]:
df = df.drop('label', axis=1)

In [12]:
df.head()

,Unnamed: 0,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,id
0,0,3.058304,-1.227988,2.570999,0.027964,0.941814,0.334969,0.160508,-1.386834,-0.749713,0.204199,0.172657,-0.801048,0
1,1,3.075964,-1.225818,2.615277,0.022178,0.941083,0.337448,-0.217198,-0.612402,-0.682841,0.089974,-0.373914,-0.506332,0
2,2,3.103364,-1.235013,2.651791,0.012594,0.944152,0.329269,0.663253,-0.498534,-0.620223,0.260127,-0.364364,-0.781249,0
3,3,3.109208,-1.244901,2.678484,0.010366,0.947364,0.319989,0.458100,-1.202168,-0.304561,0.584253,-0.922813,-0.285169,0
4,4,3.074214,-1.263514,2.661371,0.020364,0.953159,0.301783,1.347809,-0.550578,0.610944,0.626501,-1.045978,-0.063884,0


In [13]:
df = df.drop('Unnamed: 0', axis=1)
df.reset_index(inplace=True, drop=True)

In [14]:
df.drop_duplicates()

,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,id
0,3.058304,-1.227988,2.570999,0.027964,0.941814,0.334969,0.160508,-1.386834,-0.749713,0.204199,0.172657,-0.801048,0
1,3.075964,-1.225818,2.615277,0.022178,0.941083,0.337448,-0.217198,-0.612402,-0.682841,0.089974,-0.373914,-0.506332,0
2,3.103364,-1.235013,2.651791,0.012594,0.944152,0.329269,0.663253,-0.498534,-0.620223,0.260127,-0.364364,-0.781249,0
3,3.109208,-1.244901,2.678484,0.010366,0.947364,0.319989,0.458100,-1.202168,-0.304561,0.584253,-0.922813,-0.285169,0
4,3.074214,-1.263514,2.661371,0.020364,0.953159,0.301783,1.347809,-0.550578,0.610944,0.626501,-1.045978,-0.063884,0
5,2.994483,-1.301926,2.567805,0.038938,0.964072,0.262773,2.414809,1.069776,1.265800,0.580035,-1.048346,-0.200593,0
6,2.877705,-1.355284,2.426982,0.055779,0.976867,0.206445,3.261337,0.707698,1.036100,0.023216,-0.850921,-0.867426,0
7,2.595229,-1.423334,2.142191,0.076342,0.989147,0.125539,4.630348,-0.697815,1.018621,-0.063067,-0.196087,-0.033894,0
8,1.943180,-1.467603,1.509766,0.095950,0.994680,0.037479,4.045324,-1.028207,0.766063,0.011884,0.306009,0.265301,0
9,1.139627,-1.447025,0.703775,0.112156,0.992350,-0.051596,5.012080,-0.081093,0.972427,-0.363179,0.773397,0.642828,0


In [15]:
extraction_settings = ComprehensiveFCParameters()

In [16]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [17]:
X = extract_features(df,
                    column_id='id',
                    default_fc_parameters=extraction_settings,
                    impute_function= impute)

Feature Extraction: 100%|██████████| 10/10 [02:37<00:00, 15.77s/it]
 'attitude.pitch__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"'
 'attitude.pitch__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"'
 ... 'userAcceleration.z__fft_coefficient__coeff_9__attr_"imag"'
 'userAcceleration.z__fft_coefficient__coeff_9__attr_"real"'
 'userAcceleration.z__spkt_welch_density__coeff_8'] did not have any finite values. Filling with zeros.


In [18]:
X_filtered = extract_relevant_features(df, y,
                                      column_id='id',
                                      #feature_extraction_settings=extraction_settings)
                                      default_fc_parameters=extraction_settings)

Feature Extraction: 100%|██████████| 10/10 [02:36<00:00, 15.66s/it]
 'attitude.pitch__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"'
 'attitude.pitch__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"'
 ... 'userAcceleration.z__fft_coefficient__coeff_9__attr_"imag"'
 'userAcceleration.z__fft_coefficient__coeff_9__attr_"real"'
 'userAcceleration.z__spkt_welch_density__coeff_8'] did not have any finite values. Filling with zeros.


In [19]:
 X_train, X_test, X_filtered_train, X_filtered_test, y_train, y_test = train_test_split(X, X_filtered, y, test_size=.4)

cl = DecisionTreeClassifier()
cl.fit(X_train, y_train)
print(classification_report(y_test, cl.predict(X_test)))
cl.n_features_
cl2 = DecisionTreeClassifier()
cl2.fit(X_filtered_train, y_train)
print(classification_report(y_test, cl2.predict(X_filtered_test)))

             precision    recall  f1-score   support

        0.0       1.00      0.97      0.99        37
        1.0       0.98      1.00      0.99        43

avg / total       0.99      0.99      0.99        80

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00        37
        1.0       1.00      1.00      1.00        43

avg / total       1.00      1.00      1.00        80

